In [19]:
student_id = "s214527809"
student_first_last_name = "Paul Dal Pozzo"
print(student_id, student_first_last_name)

s214527809 Paul Dal Pozzo


In [87]:
# imports
import plotly, dash
from dash import Dash, html, dash_table, dcc, callback, Output, Input, ctx
import plotly.express as px
import pandas as pd

# Reading CSV file with pandas

In [63]:
# read data from csv
df = pd.read_csv('GyroscopeData.csv')

# print data
print(df.head())
print("Data rowsXcols:", df.shape)
print(type(df))

                    timestamp    x_axis    y_axis    z_axis
0  2024-08-09 06:55:19.498924  1.000000  0.045288  1.000000
1  2024-08-09 06:55:19.954485  0.652588  0.045288  1.000000
2  2024-08-09 06:55:20.399513  0.652588  0.045288 -0.571899
3  2024-08-09 06:55:21.400808  0.599854  0.045288 -0.571899
4  2024-08-09 06:55:21.987744  0.599854  0.387451 -0.571899
Data rowsXcols: (2261, 4)
<class 'pandas.core.frame.DataFrame'>


# Creating a Plotly Dash with Data

In [121]:
# Initialize the app
app = Dash()

PAGE_CURRENT: int = 1
PAGE_MAX: int = 1
DATA_ENTRIES: int = 2261

# App layout
app.layout = html.Div([
    html.H1(children='Task 6.1P - {0} - {1}'.format(student_id, student_first_last_name)),
    html.Hr(),
    html.Div([
        # drop down menu for graph types
        html.Label('Graph Type'),
        dcc.Dropdown([
            'Bar',
            'Grouped Bar', 
            'Line', 
            'Scatter', 
            'Cumulative Distribution',
            'Histpgram Average',
            'Violin',
            'Box'
        ], 'Bar', multi=False, id='graph-type'),
        html.Br(),
        # text box for entering sample size
        html.Label('Sample Size - min 20'),
        dcc.Input(value=2261, type='number', min=20, max=2261, maxLength=4, id='sample-size')
        ],
        style={'width': '25%', 'display': 'inline-block'}
    ),
    html.Div([
        # drop down menu for variable selected
        html.Label('Variable Select'),
        dcc.Dropdown(['x_axis', 'y_axis', 'z_axis'], ['x_axis', 'y_axis', 'z_axis'] , multi=True, id='variable-type')
        ],
        style={'width': '25%','vertical-align': 'top', 'display': 'inline-block'}
    ),
    html.Hr(),
    html.Div([
        dcc.Graph(figure={}, id='graph-display'),
        # buttons for navigate through data keeping the same sample size (sort of like pages)
        html.Button('Previous', id='btn-prev', n_clicks=0, style={'width': '150px'}),
        html.Label(id='information-display', style={'width': '500px'}),
        html.Button('Next', id='btn-next', n_clicks=0, style={'width': '150px'}),
    ],
        style={'padding-left': '50px'}
    ),
    html.Hr(),
    # table of current samples shown in the graph
    dash_table.DataTable(
        id='table-data',
        columns=[{'id': c, 'name': c} for c in df.columns],
        page_action='none', 
        fixed_rows={'headers': True},
        style_table={'height': '500px', 
            'overflowY': 'auto', 
            'overflowX': 'auto',
            'width': '720px'
        },
        style_cell_conditional=[
            {'if': {'column_id': 'timestamp'},
                'width': '250px', 'textAlign': 'center'},
            {'if': {'column_id': 'x_axis'},
                'width': '150px', 'textAlign': 'right'},
            {'if': {'column_id': 'y_axis'},
                'width': '150px', 'textAlign': 'right'},
            {'if': {'column_id': 'z_axis'},
                'textAlign': 'right'},
        ],
        style_header={
            'backgroundColor': 'rgb(83, 88, 120)',
            'color': 'white',
            'fontWeight': 'bold',
            'border': '1px solid black'
        },
        style_data={ 
            'border': '1px solid black',
            'backgroundColor': 'rgb(240, 240, 240)'
        },
        style_data_conditional=[
            {
                'if': {'row_index': 'even'},
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(220, 220, 220)',
            }
        ]
    ),
    html.Hr()
], 
style={'background-color': 'rgb(200, 200, 200)'}
)

def handle_sample_size_input(sample_size: int, max: int) -> int:
    if sample_size is None:
        sample_size = 20
    if sample_size > max:
        sample_size = max
    if sample_size < 20:
        sample_size = 20
    return sample_size

def calculate_pages(entries: int, current: int, slice_length: int, trigger) -> int:
    page = current
    max: int = int(entries/slice_length)
    if (entries%slice_length) > 0:
        max = max + 1

    if 'btn-prev' == trigger:
        if page > 1:
            page -= 1
    elif 'btn-next' == trigger:
        if page < max:
            page += 1
    
    global PAGE_MAX
    PAGE_MAX = max

    return page

def make_figure_for_display(data, graph_type, variable_chosen):
    WIDTH: int = 1500
    HEIGHT: int = 500
    # LINE GRAPH
    if graph_type == 'Line':
        fig = px.line(
            data,
            title=graph_type,
            x='timestamp', 
            y=variable_chosen,
            width=WIDTH, 
            height=HEIGHT,
            markers=True
        )
    # SCATTER GRAPH
    if graph_type == 'Scatter':
        fig = px.scatter(
            data,
            title=graph_type,
            x='timestamp', 
            y=variable_chosen,
            width=WIDTH, 
            height=HEIGHT
        )
    # BAR GRAPH
    if graph_type == 'Bar':
        fig = px.bar(
            data,
            title=graph_type,
            x='timestamp', 
            y=variable_chosen,
            width=WIDTH, 
            height=HEIGHT
        )
    # GROUPED BAR GRAPH
    if graph_type == 'Grouped Bar':
        fig = px.bar(
            data,
            title=graph_type,
            x='timestamp', 
            y=variable_chosen,
            width=WIDTH, 
            height=HEIGHT,
            barmode='group'
        )
    # HISTOGRAM AVERAGE GRAPH
    if graph_type == 'Histpgram Average':
        fig = px.histogram(
            data,
            title=graph_type,
            x='timestamp', 
            y=variable_chosen,
            width=WIDTH, 
            height=HEIGHT,
            histfunc='avg'
        )
    # VIOLIN GRAPH
    if graph_type == 'Violin':
        fig = px.violin(
            data,
            title=graph_type,
            y=variable_chosen,
            width=WIDTH, 
            height=HEIGHT
        )
    # ECDF GRAPH
    if graph_type == 'Cumulative Distribution':
        fig = px.ecdf(
            data,
            title=graph_type,
            x=variable_chosen,
            width=WIDTH, 
            height=HEIGHT
        )
    # BOX GRAPH
    if graph_type == 'Box':
        fig = px.box(
            data,
            title=graph_type,
            y=variable_chosen,
            points='all',
            width=WIDTH, 
            height=HEIGHT
        )
    fig.update_layout(plot_bgcolor = "rgb(250,250,250)", paper_bgcolor = "rgb(200, 200, 200)")
    return fig

@callback(
    Output(component_id='graph-display', component_property='figure'),
    Output(component_id='table-data', component_property='data'),
    Output(component_id='information-display', component_property='children'),
    Input(component_id='graph-type', component_property='value'),
    Input(component_id='sample-size', component_property='value'),
    Input(component_id='variable-type', component_property='value'),
    Input(component_id='btn-prev', component_property='n_clicks'),
    Input(component_id='btn-next', component_property='n_clicks')
)
def update_graph(graph_type_chosen, sample_size, variable_chosen, prev, next):
    fig = {}
    if len(variable_chosen) == 0:
        return fig
    
    global PAGE_CURRENT
    global DATA_ENTRIES
    global PAGE_MAX

    slice_length: int = handle_sample_size_input(sample_size, DATA_ENTRIES)

    page: int = calculate_pages(DATA_ENTRIES, PAGE_CURRENT, slice_length, ctx.triggered_id)
    PAGE_CURRENT = page

    start: int = ((page - 1) * slice_length)
    end: int = start + slice_length
    if end > DATA_ENTRIES:
        end = DATA_ENTRIES
        start = DATA_ENTRIES - slice_length
    
    data = df.iloc[start:end]
    msg: str
    msg = 'Entries: {0} to {1}. Page: {2}/{3}'.format(start, end, page, PAGE_MAX)

    return make_figure_for_display(data, graph_type_chosen, variable_chosen), data.to_dict('records'), msg

# Run the app
if __name__ == '__main__':
    app.run(debug=True, jupyter_mode="tab")

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>